# Agricultural Dedication Analysis for Hawaii Counties

This notebook provides analysis of agricultural property tax dedications across Hawaii counties.

## Setup

In [ ]:
# Import required libraries
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from ag_dedicated import config
from ag_dedicated.extractors.pdf_extractor import PDFExtractor
from ag_dedicated.analysis.statute_comparison import StatuteComparison

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Dedication Data

Load the processed dedication data from CSV files.

In [ ]:
# Load cleaned dedication data
output_dir = config.get_path('paths.output')
cleaned_path = output_dir / 'cleaned_output.csv'

if cleaned_path.exists():
    dedications = pd.read_csv(cleaned_path)
    print(f"Loaded {len(dedications):,} dedication records")
    print(f"\nColumns: {list(dedications.columns)}")
    dedications.head()
else:
    print(f"Data file not found at {cleaned_path}")
    print("Run: ag-dedicated extract")

## 2. Data Overview

Examine the structure and quality of the data.

In [ ]:
# Basic statistics
print("Dataset shape:", dedications.shape)
print("\nData types:")
print(dedications.dtypes)
print("\nMissing values:")
print(dedications.isnull().sum())
print("\nBasic statistics:")
dedications.describe()

## 3. Temporal Analysis

Analyze trends over time.

In [ ]:
# Dedications by year
if 'Year' in dedications.columns:
    year_counts = dedications['Year'].value_counts().sort_index()
    
    plt.figure(figsize=(14, 6))
    year_counts.plot(kind='bar', color='steelblue')
    plt.title('Agricultural Dedications by Year - Honolulu County', fontsize=16, fontweight='bold')
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Dedications', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nDedications by Year:")
    print(year_counts)

## 4. County Statute Comparison

Compare dedication programs across all Hawaii counties.

In [ ]:
# Generate comparison
comparison = StatuteComparison(config)

# County summaries
summary_df = comparison.compare_all_counties()
print("County Program Summary:")
display(summary_df)

# Dedication periods
periods_df = comparison.compare_dedication_periods()
print("\nDedication Period Options:")
display(periods_df)

In [ ]:
# Full comparison report
report = comparison.generate_comparison_report()
print(report)

## 5. Dedication Period Analysis

Analyze the distribution of dedication periods (if data available).

In [ ]:
# Check for end year column
end_year_cols = [col for col in dedications.columns if 'end' in col.lower() and 'year' in col.lower()]

if end_year_cols:
    end_year_col = end_year_cols[0]
    print(f"Analyzing dedication end years from column: {end_year_col}")
    
    # Distribution of end years
    plt.figure(figsize=(12, 6))
    dedications[end_year_col].value_counts().sort_index().plot(kind='bar', color='coral')
    plt.title('Distribution of Dedication End Years', fontsize=16, fontweight='bold')
    plt.xlabel('End Year', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No 'End Year' column found in data")

## 6. Geographic Distribution (TMK Analysis)

Analyze the geographic distribution of dedications based on TMK zones.

In [ ]:
# Extract zone from TMK if available
if 'TMK' in dedications.columns:
    # TMK format is typically zone-section-plat-parcel
    dedications['TMK_Zone'] = dedications['TMK'].astype(str).str.split('-').str[0]
    
    zone_counts = dedications['TMK_Zone'].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    zone_counts.plot(kind='barh', color='seagreen')
    plt.title('Top 10 TMK Zones by Number of Dedications', fontsize=16, fontweight='bold')
    plt.xlabel('Number of Dedications', fontsize=12)
    plt.ylabel('TMK Zone', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("\nTop zones:")
    print(zone_counts)

## 7. Export Analysis Results

Save processed data and analysis results.

In [ ]:
# Export year summary
if 'Year' in dedications.columns:
    year_summary = dedications.groupby('Year').size().reset_index(name='Count')
    
    output_path = config.data_dir / 'yearly_summary.csv'
    year_summary.to_csv(output_path, index=False)
    print(f"Saved yearly summary to {output_path}")

# Export statute comparison
comparison_dir = config.data_dir / 'statute_comparison'
comparison.export_to_csv(comparison_dir)
print(f"Saved statute comparison to {comparison_dir}")

## Next Steps

1. **Scrape parcel data**: Use the CLI to scrape detailed parcel information
2. **Tax benefit analysis**: Calculate actual tax savings per dedication
3. **Compliance monitoring**: Integrate imagery analysis for land use verification
4. **Multi-county expansion**: Process dedications from Hawaii, Maui, and Kauai counties